In [ ]:
import pandas as pd
from plotnine import *
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import keras
from keras.layers import *
from keras.models import Model
%matplotlib inline

## Load data

In [ ]:
import os
os.system('kaggle datasets download -d uciml/pima-indians-diabetes-database -f diabetes.csv');

In [ ]:
dataset_df = pd.read_csv('diabetes.csv')
dataset_df.head()

## Preprocess data

In [ ]:
means = dataset_df.mean(axis=0)
dataset_df_p = dataset_df - dataset_df.mean(axis=0)
dataset_df_p /= dataset_df_p.max(axis=0)

## Select heathy people data

In [ ]:
df_healthy = dataset_df_p[dataset_df_p.Outcome < 0]

In [ ]:
df_healthy.head()

In [ ]:
REMOVE_FEATURES = ['BMI', 'Outcome']
FEATURES = [feature for feature in df_healthy.columns if feature not in REMOVE_FEATURES]
N_FEATURES = len(FEATURES)

In [ ]:
x_train = df_healthy[FEATURES].values
y_train = df_healthy[[REMOVE_FEATURES[0]]].values

## Create an ensemble of models

In [ ]:
def create_base_model(n_hidden):
    i = Input((N_FEATURES, ))
    h = Dense(n_hidden, kernel_initializer='normal', activation='relu')(i)
    o = Dense(1, kernel_initializer='normal')(h)
    return Model(i, o)

In [ ]:
models = []
N_MODELS = 5
n_hidden_neurons = N_FEATURES ** 2
for _ in range(N_MODELS):
    models.append(create_base_model(32))

In [ ]:
models[0].summary()

## Train ensemble

In [ ]:
def train_model(model, input, target):
    model.compile(loss='mean_squared_error', optimizer=keras.optimizers.Adam())
    model.fit(input, target, epochs=150, batch_size=1000)

In [ ]:
for i in range(N_MODELS):
    train_model(models[i], x_train, y_train)

In [ ]:
train_model(models[0], x_train, y_train)

Done!